# **Uploading the files and concating the DFs**

In [2]:
import pandas as pd

In [3]:
letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','z']

In [4]:
#df_a = pd.read_csv("../BEEV/letter_scraped/"+ 'a' +"_scraped.csv")                #Remember to ask Cristina about yellow stuff after backslash
#df_a

In [5]:
#Building a dicto_scraped with all DF to be able to concat everything
dicto_scraped = {letter : pd.read_csv("../BEEV/letter_scraped/"+ letter +"_scraped.csv",index_col=None) for letter in letters}

/var/folders/zp/g__9z3d17bl6nkkpnxb0p46r0000gn/T/ipykernel_6089/1819279109.py:2: DtypeWarning: Columns (56,80,95,110,111,112,116,121,122,123,127,128,129,130,131,132,133,134,135,136,137,138,139,142) have mixed types.Specify dtype option on import or set low_memory=False.
  dicto_scraped = {letter : pd.read_csv("../BEEV/letter_scraped/"+ letter +"_scraped.csv",index_col=None) for letter in letters}
/var/folders/zp/g__9z3d17bl6nkkpnxb0p46r0000gn/T/ipykernel_6089/1819279109.py:2: DtypeWarning: Columns (55,56,108,118,119,123,124,125,126,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,149,150,151,152,153,154,155,156,157,158,159,160,167,168,169,170,171) have mixed types.Specify dtype option on import or set low_memory=False.
  dicto_scraped = {letter : pd.read_csv("../BEEV/letter_scraped/"+ letter +"_scraped.csv",index_col=None) for letter in letters}


In [6]:
#Building a list of shape to check coherence between letter.csv and letter_scarped_csv
scraped_shape = [dicto_scraped[let].shape[0] for let in dicto_scraped]

In [7]:
#Building a dicto_csv with all url list to be able to compare with dicto_scraped
dicto_csv = {letter : pd.read_csv("../BEEV/letter_csvs/"+ letter +".csv",index_col=None) for letter in letters}

In [8]:
#Building a list of shape to check coherence between letter.csv and letter_scarped_csv
csv_shape = [dicto_csv[lett].shape[0] for lett in dicto_csv.keys()]

In [9]:
#Calculating the difference between both lenght list
scraped_shape
for i in range(len(scraped_shape)):
    print(scraped_shape[i] - csv_shape[i])  

-14
-13
-14
-6
0
-21
-5
-15
-3
-4
-7
-7
-35
-11
-3
-7
0
-7
-10
-18
0
-10
-7
0
0


In [10]:
#The difference come from the empty dataframe we scrapped so we have to count the number of empty DF to see if it macth
# test calculation --->  
#dicto_csv['a']['dataframe'][dicto_csv['a']['dataframe'].str.contains('Empty') == True].shape[0]
empty_df_list = [dicto_csv[letter]['dataframe'][dicto_csv[letter]['dataframe'].str.contains('Empty') == True].shape[0] for letter in letters]
    

In [11]:
#Calculating the difference between both lenght list and the number of empty DF
# If every number is equal to 0 we are ok ! next step will be to check the model coherence
for i in range(len(scraped_shape)):
    print(scraped_shape[i] - csv_shape[i] + empty_df_list[i])  

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


# **GEV dataframe**

In [12]:
#Building the DF using the DF in the dicto_scraped
df = pd.concat([dicto_scraped[letter] for letter in letters])

In [13]:
df_gev = df[df['Powertrain Architecture'] == 'Internal Combustion engine']
df_gev

,Brand,Model,Generation,Modification (Engine),Start of production,Powertrain Architecture,Body type,Seats,Doors,Performance specs,...,200 km/h - 0,Acceleration 0 - 100 km/h (CNG),Maximum speed (CNG),CO2 emissions (CNG) (WLTP),"Fuel consumption (economy) - urban (CNG) (NEDC, WLTP equivalent)","Fuel consumption (economy) - extra urban (CNG) (NEDC, WLTP equivalent)","Fuel consumption (economy) - combined (CNG) (NEDC, WLTP equivalent)","CO2 emissions (CNG) (NEDC, WLTP equivalent)",Acceleration 0 - 100 km/h (Ethanol - E85),Maximum speed (Ethanol - E85)
0,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp) Automatic,2018 year,Internal Combustion engine,Roadster,2,2,Performance specs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp),2018 year,Internal Combustion engine,Roadster,2,2,Performance specs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp) Automatic,2016 year,Internal Combustion engine,Cabriolet,2,2,Performance specs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp),2016 year,Internal Combustion engine,Cabriolet,2,2,Performance specs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Abarth,500,500C,1.4 T-Jet (140 Hp) MTA,2010 year,Internal Combustion engine,Cabriolet,4,2,Performance specs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,ZX,GrandTiger,GrandTiger,2.4 (126 Hp) 4x4,2007 year,Internal Combustion engine,Pick-up,5.0,4,Performance specs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,ZX,GrandTiger,GrandTiger,2.4 (126 Hp) 4x2,2007 year,Internal Combustion engine,Pick-up,5.0,4,Performance specs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,ZX,Landmark,Landmark,2.4 (126 Hp) 4x4,2006 year,Internal Combustion engine,SUV,7.0,5,Performance specs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,ZX,Landmark,Landmark,2.4 (126 Hp) 4x2 Automatic,2006 year,Internal Combustion engine,SUV,7.0,5,Performance specs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#Definition of the key word for filtering columns
words_wanted4_column = ['Brand','Model','Generation', 'Engine', 'production','Powertrain','Body','Fuel','CO2','Power']


In [15]:
#Checking manually to see which key words we need for the columns
#df_gev.columns[79:110]
len(list(df_gev.columns))

192

In [16]:
#Building the list of columns containing the key words
col_filtered = []
for word in words_wanted4_column:
  for w in list(df_gev.columns):
    if w.lower().find(word.lower()) != -1:
      col_filtered.append(w)
len(col_filtered)

94

In [123]:
col_filtered

['Brand',
 'Model',
 'Engine Model/Code',
 'Generation',
 'Modification (Engine)',
 'Engine specs',
 'Engine location',
 'Engine displacement',
 'Engine aspiration',
 'Engine oil capacity',
 'Engine Model/Code',
 'Engine oil specification',
 'Maximum engine speed',
 'Engine systems',
 'Internal combustion engine specs',
 'Start of production',
 'End of production',
 'Powertrain Architecture',
 'Body type',
 'Fuel consumption (economy) - urban',
 'Fuel consumption (economy) - extra urban',
 'Fuel consumption (economy) - combined',
 'Fuel Type',
 'Fuel tank capacity',
 'Fuel consumption (economy) - urban (NEDC)',
 'Fuel consumption (economy) - extra urban (NEDC)',
 'Fuel consumption (economy) - combined (NEDC)',
 'Fuel System',
 'Fuel consumption at Low speed (WLTP)',
 'Fuel consumption at Medium speed (WLTP)',
 'Fuel consumption at high speed (WLTP)',
 'Fuel consumption at very high speed (WLTP)',
 'Combined fuel consumption (WLTP)',
 'Fuel consumption (economy) - urban (LPG)',
 'Fuel c

In [23]:
df_gev[['Fuel consumption (economy) - urban',
 'Fuel consumption (economy) - extra urban',
 'Fuel consumption (economy) - combined',
 'Fuel consumption (economy) - urban (NEDC)',
 'Fuel consumption (economy) - extra urban (NEDC)',
 'Fuel consumption (economy) - combined (NEDC)',
 'Fuel consumption at Low speed (WLTP)',
 'Fuel consumption at Medium speed (WLTP)',
 'Fuel consumption at high speed (WLTP)',
 'Fuel consumption at very high speed (WLTP)',
 'Combined fuel consumption (WLTP)',
 'Fuel consumption (economy) - urban (LPG)',
 'Fuel consumption (economy) - extra urban (LPG)',
 'Fuel consumption (economy) - combined (LPG)',
 'Fuel consumption (economy) - urban (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - extra urban (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - combined (NEDC, WLTP equivalent)',
 'Combined fuel consumption (WLTP) (CNG)',
 'Fuel consumption (economy) - urban (CNG) (NEDC)',
 'Fuel consumption (economy) - extra urban (CNG) (NEDC)',
 'Fuel consumption (economy) - combined (CNG) (NEDC)',
 'Fuel consumption (economy) - urban (CNG)',
 'Fuel consumption (economy) - extra urban (CNG)',
 'Fuel consumption (economy) - combined (CNG)',
 'Fuel consumption (economy) - urban (Ethanol - E85)',
 'Fuel consumption (economy) - extra urban (Ethanol - E85)',
 'Fuel consumption (economy) - combined (Ethanol - E85)',
 'Fuel consumption (economy) - urban (LPG) (NEDC)',
 'Fuel consumption (economy) - extra urban (LPG) (NEDC)',
 'Fuel consumption (economy) - combined (LPG) (NEDC)',
 'Fuel consumption (economy) - urban (LPG) (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - extra urban (LPG) (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - combined (LPG) (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - urban (CNG) (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - extra urban (CNG) (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - combined (CNG) (NEDC, WLTP equivalent)']]#.isna().sum()

,Fuel consumption (economy) - urban,Fuel consumption (economy) - extra urban,Fuel consumption (economy) - combined,Fuel consumption (economy) - urban (NEDC),Fuel consumption (economy) - extra urban (NEDC),Fuel consumption (economy) - combined (NEDC),Fuel consumption at Low speed (WLTP),Fuel consumption at Medium speed (WLTP),Fuel consumption at high speed (WLTP),Fuel consumption at very high speed (WLTP),...,Fuel consumption (economy) - combined (Ethanol - E85),Fuel consumption (economy) - urban (LPG) (NEDC),Fuel consumption (economy) - extra urban (LPG) (NEDC),Fuel consumption (economy) - combined (LPG) (NEDC),"Fuel consumption (economy) - urban (LPG) (NEDC, WLTP equivalent)","Fuel consumption (economy) - extra urban (LPG) (NEDC, WLTP equivalent)","Fuel consumption (economy) - combined (LPG) (NEDC, WLTP equivalent)","Fuel consumption (economy) - urban (CNG) (NEDC, WLTP equivalent)","Fuel consumption (economy) - extra urban (CNG) (NEDC, WLTP equivalent)","Fuel consumption (economy) - combined (CNG) (NEDC, WLTP equivalent)"
0,9.1 l/100 km 25.85 US mpg31.04 UK mpg10.99 km/l,5.2 l/100 km 45.23 US mpg54.32 UK mpg19.23 km/l,6.6 l/100 km 35.64 US mpg42.8 UK mpg15.15 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8.5 l/100 km 27.67 US mpg33.23 UK mpg11.76 km/l,5.1 l/100 km 46.12 US mpg55.39 UK mpg19.61 km/l,6.4 l/100 km 36.75 US mpg44.14 UK mpg15.63 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9.1 l/100 km 25.85 US mpg31.04 UK mpg10.99 km/l,5.1 l/100 km 46.12 US mpg55.39 UK mpg19.61 km/l,6.6 l/100 km 35.64 US mpg42.8 UK mpg15.15 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8.5 l/100 km 27.67 US mpg33.23 UK mpg11.76 km/l,5.1 l/100 km 46.12 US mpg55.39 UK mpg19.61 km/l,6.4 l/100 km 36.75 US mpg44.14 UK mpg15.63 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,8.4 l/100 km 28 US mpg33.63 UK mpg11.9 km/l,5.4 l/100 km 43.56 US mpg52.31 UK mpg18.52 km/l,6.5 l/100 km 36.19 US mpg43.46 UK mpg15.38 km/l,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,NaN,NaN,13 l/100 km 18.09 US mpg21.73 UK mpg7.69 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,NaN,NaN,13 l/100 km 18.09 US mpg21.73 UK mpg7.69 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,NaN,12 l/100 km 19.6 US mpg23.54 UK mpg8.33 km/l,13 l/100 km 18.09 US mpg21.73 UK mpg7.69 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,9 l/100 km 26.13 US mpg31.39 UK mpg11.11 km/l,13 l/100 km 18.09 US mpg21.73 UK mpg7.69 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
df_gev.shape

(43112, 192)

In [10]:
col_list = []
for col in df_gev.columns:
    col_list += [col]

In [141]:
#df_gev.iloc[54]
dicto_csv['a'].iloc[54]

Unnamed: 0                                                          54
brands_url                     https://www.auto-data.net/en/ac-brand-1
models                                                  ac-ace-model-1
models_url                 https://www.auto-data.net/en/ac-ace-model-1
generations                                   ac-ace-i-generation-8637
generations_url      https://www.auto-data.net/en/ac-ace-i-generati...
modifications                                                      NaN
modifications_url                     https://www.auto-data.net/en/nan
dataframe                      Empty DataFrame\nColumns: []\nIndex: []
Name: 54, dtype: object

In [11]:
[df_gev.columns.str_contains('NEDC') for  in col_list]

SyntaxError: invalid syntax (Temp/ipykernel_6784/2109108790.py, line 1)

# **EV dataframe**

In [ ]:
df['Powertrain Architecture'].value_counts()

Internal Combustion engine                                                                          43112
MHEV (Mild Hybrid Electric Vehicle, power-assist hybrid, battery-assisted hybrid vehicles, BAHV)      645
BEV (Electric Vehicle)                                                                                418
PHEV (Plug-in Hybrid Electric Vehicle)                                                                326
FHEV (Full Hybrid Electric Vehicle)                                                                   314
FCEV (Fuel Cell Electric Vehicle)                                                                       7
PFCEV (Plug-in Fuel Cell Electric Vehicle)                                                              1
Name: Powertrain Architecture, dtype: int64

In [ ]:
condition = df['Powertrain Architecture'] == 'Internal Combustion engine'
df_electric = df[~condition]                                             #Makes no sense why it doesn't work with the above written

In [ ]:
# #This is the dataframe with all the information we want
# page4 = 'https://www.auto-data.net/en/alfa-romeo-mito-1.6-jtdm-120hp-16680'
# what = pd.read_html(page4)[1]  
# what

# what2 = what[(what['General information'] == 'Brand')
# |(what['General information'] == 'Model')
# |(what['General information'] == 'Generation')
# |(what['General information'] == 'Modification (Engine)')
# |(what['General information'] == 'CO2 emissions') 
# |(what['General information'] == 'Body type')
# |(what['General information'] == 'Fuel tank capacity')
# |(what['General information'] == 'Fuel consumption (economy) - combined')
# |(what['General information'] == 'Fuel Type')
# |(what['General information'] == 'Power')]
# what2.index = what2['General information']
# what3 = what2.T
# what3.drop(axis = 0, labels = 'General information', inplace = True)
# what3['Fuel tank(l)'] = what3['Fuel tank capacity'].apply(lambda x : x.split(' ')[0])
# what3['Fuel consumption(l/100km)'] = what3['Fuel consumption (economy) - combined'].apply(lambda x : x.split(' ')[0])
# what3['Power(Hp)'] = what3['Power'].apply(lambda x: x.split(' ')[0])
# what3['Range (Km)'] = what3[['Fuel tank(l)','Fuel consumption(l/100km)']].apply(lambda item: int(item[0])/float(item[1])*100,axis=1)

# what4 = what3
# what4.drop(columns = ['Fuel consumption (economy) - combined', 'Fuel tank capacity'], inplace = True)
# what4                               #We need to remove this General information column still